[Rendered Version](http://nbviewer.jupyter.org/github/MaxGhenis/taxcalc-notebooks/blob/master/taxcalc%20Training%20Doc%20-%20TCJA%20-%20Tedeschi%20weights.ipynb.ipynb)

This resembles the [normal training notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/Training%20Doc.ipynb), with the following changes:
* Models the House TCJA reform, not the sample reform.
* Uses Ernie Tedeschi's weights instead. This approach can be used to use any other weight file.
* Removes reference to sample reform dictionary.

In [1]:
from taxcalc import *

This example uses the packaged CPS file. If you were to use the PUF, you would create the records class using
`Records()` or `Records('path to puf')` if the PUF is not in your current working directory.

In [2]:
# Initiate baseline calculator
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
# Advance and calculate.
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [3]:
# Initiate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


You can specify your reforms in a JSON file and, using the `read_json_param_objects` method, create a dictionary containing both policy reforms and behavioral assumptions

The policy reform and any behavioral assumptions need to be in separate JSON files.

Here is [more information](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/reforms/REFORMS.md) on creating reform files.

In [4]:
reforms = calc_x.read_json_param_objects('TCJA_House_Amended.json', 'JCT_Behavior.json')
reforms

{'behavior': {2018: {u'_BE_inc': [-0.25]}},
 'consumption': {2018: {u'_MPC_e19800': [0.2]}},
 'growdiff_baseline': {},
 'growdiff_response': {},
 'policy': {2018: {u'_ALD_DomesticProduction_hc': [1],
   u'_ALD_StudentLoan_hc': [1],
   u'_ALD_Tuition_hc': [1],
   u'_AMT_rt1': [0],
   u'_AMT_rt2': [0],
   u'_CG_brk1': [[38600, 77200, 38600, 51700, 77200]],
   u'_CG_brk2': [[425800, 479000, 239500, 452400, 479000]],
   u'_CR_SchR_hc': [1],
   u'_CTC_c': [1100],
   u'_CTC_ps': [[115000, 230000, 115000, 115000, 115000]],
   u'_DependentCredit_Child_c': [500],
   u'_DependentCredit_Nonchild_c': [300],
   u'_FilerCredit_c': [[300, 600, 300, 300, 300]],
   u'_ID_Casualty_hc': [1],
   u'_ID_Medical_hc': [1],
   u'_ID_Miscellaneous_hc': [1],
   u'_ID_RealEstate_c': [[10000, 10000, 10000, 10000, 10000]],
   u'_ID_StateLocalTax_hc': [1],
   u'_ID_prt': [0],
   u'_II_brk1': [[45000, 90000, 45000, 67500, 90000]],
   u'_II_brk2': [[200000, 260000, 130000, 200000, 260000]],
   u'_II_brk3': [[500000, 1

In [5]:
calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [6]:
calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)

AssertionError: 

### Viewing the results

In [ ]:
from taxcalc.utils import *

#### Analyzing Individual Variables

Individual variables are attributes of the records class and can therefore be accessed using a simple dot notation

In [ ]:
baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

#### Diagnostic Table

Diagnostic tables are the most straight forward methods of evaluation. They simply show aggregate values for a given calculator

In [ ]:
create_diagnostic_table(calc)

In [ ]:
create_diagnostic_table(calc_response)

#### Distribution Table

The distribution table shows the same information as the diagnostic table, but broken down by income bin or decile. You can view the results as either the weighted average or the weighted sum in each bin

In [ ]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_avg')

In [ ]:
create_distribution_table(calc_response.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_sum')

#### Differences Table

The differences table displays the difference between your baseline and refoms. You can also group the results by decile or income bin.

In [ ]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

#### Plotting

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

In [ ]:
mtr_plot_data = mtr_graph_data(calc, calc_response)

In [ ]:
show(xtr_graph_plot(mtr_plot_data))

In [ ]:
atr_plot_data = atr_graph_data(calc, calc_response)

In [ ]:
show(xtr_graph_plot(atr_plot_data))

#### Multiyear diagnostic tables

You can also produce diagnostic tables up to 2026

In [ ]:
multiyear_diagnostic_table(calc, num_years=9)

In [ ]:
multiyear_diagnostic_table(calc_response, num_years=9)

#### Reporting your changes

To display what reforms you included in a way that is easy for humans to read, you can use the `reform_documentation` method. It will print out all of the policy parameters you've specified with a short description and their default and current values


_Note: this feature is not yet available in the taxcalc package. You must use the source code to access it. It will be available in the next taxcalc release._

In [ ]:
print(calc_response.reform_documentation(reforms))

One thing I didn't cover is how to extract the marginal tax rates. All you need to do is call the `Calculator.mtr()` method. It will return MTR for individual income tax, payroll tax, and individaul income tax + payroll tax

In [ ]:
mtr_payroll, mtr_income, mtr_combined = calc.mtr()

In [ ]:
mtr_payroll

In [ ]:
mtr_income

In [ ]:
mtr_combined